In [1]:
#Importamos librerias:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
#Agregamos la ruta de la carpeta functions:
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))

                        **Extracción y Transformación Movies**                                                             

In [3]:
# Creamos el DataFrame y leemos el archivo:
movies_df = pd.read_csv('../data/movies_dataset.csv')

C:\Users\Julia\AppData\Local\Temp\ipykernel_10004\56624348.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('../data/movies_dataset.csv')


In [4]:
# Importamos la función y convertimos la columna a float o númerico:
from convertidor_float import convertir_float

movies_df = convertir_float(movies_df, 'popularity')

In [5]:
# Dropeamos las filas con valores nulos en 'popularity'
movies_df.dropna(subset=['popularity'], inplace=True)

In [6]:
#Los valores nulos del campo release date deben eliminarse:
movies_df.dropna(subset=['release_date'], inplace= True)

In [7]:
# Transformamos a parquet:
movies_df.to_parquet('../data/movies_dataset.parquet', engine='pyarrow')

In [8]:
# Eliminar las columnas que no serán utilizadas:
movies_df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [9]:
#Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0:
movies_df['revenue'].fillna(0, inplace=True)
movies_df['budget'].fillna(0, inplace=True)

C:\Users\Julia\AppData\Local\Temp\ipykernel_10004\1524035447.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['revenue'].fillna(0, inplace=True)
C:\Users\Julia\AppData\Local\Temp\ipykernel_10004\1524035447.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [10]:
# Borramos los valores Nan de la columna Overview:
movies_df.dropna(subset='overview', inplace=True)

In [11]:
# Observamos la cantidad de filas en overview que están vacias:
movies_df[movies_df['overview'].str.strip() == ""].shape[0]

5

In [12]:
# Procedemos a borrarlas:
movies_df = movies_df[movies_df['overview'].str.strip() != ""]

In [13]:
# Observamos el tipo de dato de la columa 'release_date'
movies_df['release_date'].dtype

dtype('O')

In [14]:
# Expresión regular para verificar el formato "AAAA-mm-dd":
regex = r'^\d{4}-\d{2}-\d{2}$'

# Verificar si las fechas cumplen con el formato:
datos_validos = movies_df['release_date'].str.match(regex)

# Filtrar las filas que no cumplen con el formato y obtener los índices:
datos_invalidos = movies_df[~datos_validos].index

In [15]:
# Borramos los datos que no cumplan con las normas "AAAA-mm-dd":
movies_df = movies_df.drop([datos_invalidos], errors='ignore')

In [16]:
# Ahora si transformamos los datos de la columna 'release_date' to_datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

In [17]:
# Confirmamos la transformación
print(movies_df['release_date'].dtype)

datetime64[ns]


In [18]:
# Creamnos la columna release_year donde se extraerán el año de la fecha de estreno
movies_df['release_year'] = movies_df['release_date'].dt.year

In [19]:
# Confirmamos la creación:
movies_df[['release_year', 'release_date']].head(3)

,release_year,release_date
0,1995,1995-10-30
1,1995,1995-12-15
2,1995,1995-12-22


In [20]:
# Observamos el tipo de dato para las columnas 'revenue' y 'budget':
print(movies_df['revenue'].dtype, movies_df['budget'].dtype)

float64 object


In [21]:
#Pasamos la columna budget a numerico así podemos operacionalizarla.
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')

In [22]:
#Creo la columna con el ROI de las peliculas
movies_df['return'] = np.where(movies_df['budget'] > 0, 
                                movies_df['revenue'] / movies_df['budget'] ,0)

                                **Desanidar columnas**                          

                **Comenzamos con la la columna belong_to_collection**               

In [23]:
# Observamos los tipos de datos que conforman a la columna belongs_to_collection
movies_df['belongs_to_collection'].apply(type).value_counts()

belongs_to_collection
<class 'float'>    40001
<class 'str'>       4429
Name: count, dtype: int64

In [24]:
#Importamos la función para generar la conversión y desanidar:
from convertidor_list_dict import convertir_list_dict

In [25]:
# Convertimos la columna dict:
movies_df['belongs_to_collection'] = movies_df['belongs_to_collection'].apply(convertir_list_dict)

In [26]:
# Confirmamos la transformación:
movies_df['belongs_to_collection'].apply(type).value_counts()

belongs_to_collection
<class 'NoneType'>    40001
<class 'dict'>         4429
Name: count, dtype: int64

In [27]:
# Desanidamos y normalizamos la columna y creamos un df que contiene la columna 
# belong_to_coleccion desanidada y normalizada:
btc = pd.json_normalize(movies_df['belongs_to_collection'])

In [28]:
# Cambiamos los nombres de las columnas del DataFrame recientemente creado:
btc = btc.rename(columns={'id': 'btc_id', 'name': 'btc_name', 
                            'poster_path': 'btc_poster_path', 
                            'backdrop_path': 'btc_backdrop_path'})

In [29]:
# Procedemos a concatenar el DataFrame orginal con el que creamos:
movies_df = pd.concat([movies_df, btc], axis=1)

In [30]:
# Borramos las columnas concatenadas que no nos sirven para el analisis
movies_df.drop(columns=['btc_poster_path', 'btc_backdrop_path'], axis=1, inplace=True)
# Al ya estar desanidada procedemos a borrarla:
movies_df.drop(columns='belongs_to_collection', axis=1, inplace=True)

In [31]:
# Hacemos lo mismo para el DataFrame btc
btc.drop(columns=['btc_poster_path', 'btc_backdrop_path'], axis=1, inplace=True)

                        **Continuamos con la columna genres**                   

In [32]:
# Observamos los tipos de datos que conforman a la columna genres:
movies_df['genres'].apply(type).value_counts()

genres
<class 'str'>      44430
<class 'float'>      986
Name: count, dtype: int64

In [33]:
# Convertimos la columna a list:
movies_df['genres'] = movies_df['genres'].apply(convertir_list_dict)

In [34]:
# Dropeamos aquellas filas donde se contienen valores None en la columna
movies_df = movies_df[movies_df['genres'].notna()]

In [35]:
# Confirmamos la transformación:
movies_df['genres'].apply(type).value_counts()

genres
<class 'list'>    44430
Name: count, dtype: int64

In [36]:
# Importamos la función para desnidar la columna:
from desanidar import desanidar_columna

In [37]:
# Desanidamos y normalizamos la columna y creamos un df que contiene la columna 
# genres desanidada y normalizada:
genres = desanidar_columna(movies_df, 'genres','genres')

In [38]:
# Al ya estar desanidada procedemos a borrarla:
movies_df.drop(columns='genres', axis=1, inplace=True)

                        **Continuamos con production_companies**                          

In [39]:
# Observamos los tipos de datos que conforman a la columna production_companies:
movies_df['production_companies'].apply(type).value_counts()

production_companies
<class 'str'>    44430
Name: count, dtype: int64

In [40]:
# Convertimos la columna a list:
movies_df['production_companies'] = movies_df['production_companies'].apply(convertir_list_dict)

In [41]:
# Confirmamos la transformación:
movies_df['production_companies'].apply(type).value_counts()

production_companies
<class 'list'>    44430
Name: count, dtype: int64

In [42]:
# Desanidamos y normalizamos la columna y creamos un df que contiene la columna 
# production_companies desanidada y normalizada:
production_companies = desanidar_columna(movies_df, 'production_companies', 'production_companies')

In [43]:
# No nos sirve tener dos id, procedemos a borrarlos:
production_companies.drop(columns='id', inplace=True)

In [44]:
# Observamos que tiene muchos NaN:
production_companies['name'].isna().sum()

np.int64(11241)

In [45]:
# Procedemos a borrar las filas que tienen NaNs para la columna name
production_companies.dropna(subset='name', inplace=True)

In [46]:
# Al ya estar desanidada procedemos a borrarla:
movies_df.drop(columns='production_companies', axis=1, inplace=True)

                        **Continuamos con production_countries**                              

In [47]:
# Observamos los tipos de datos que conforman a la columna production_companies:
movies_df['production_countries'].apply(type).value_counts()

production_countries
<class 'str'>    44430
Name: count, dtype: int64

In [48]:
# Convertimos la columna a list:
movies_df['production_countries'] = movies_df['production_countries'].apply(convertir_list_dict)

In [49]:
# Confirmamos la transformación:
movies_df['production_countries'].apply(type).value_counts()

production_countries
<class 'list'>    44430
Name: count, dtype: int64

In [50]:
# Desanidamos y normalizamos la columna y creamos un df que contiene la columna 
# production_countries desanidada y normalizada:
production_countries = desanidar_columna(movies_df, 'production_countries', 
                                        'production_countries_desanidado')

In [51]:
# Al ya estar desanidada procedemos a borrarla:
movies_df.drop(columns='production_countries', axis=1, inplace=True)

                            **Finalizamos con spoken_langueges**                                

In [52]:
# Observamos los tipos de datos que conforman a la columna production_companies:
movies_df['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'str'>    44430
Name: count, dtype: int64

In [53]:
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(convertir_list_dict)

In [54]:
# Confirmamos transformación
movies_df['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'list'>    44430
Name: count, dtype: int64

In [55]:
# Desanidamos y normalizamos la columna y creamos un df que contiene la columna 
# production_countries desanidada y normalizada:
spoken_languages = desanidar_columna(movies_df, 'spoken_languages', 
                                    'spoken_languages_desanidada')

In [56]:
# Al ya estar desanidada procedemos a borrarla:
movies_df.drop(columns='spoken_languages', axis=1, inplace=True)

                            **Algunos detalles...**                                    

In [57]:
# Tomaremos para el analisis únicamente aquellas peliculas que fueron realizas
movies_df = movies_df[movies_df['status'] == 'Released']
# Al trabajar unicamente con peliculas ya hechas, la columna status no tiene sentido
movies_df.drop(columns='status', axis=1, inplace=True)

In [58]:
# El tagline no es un material que aporte al analisis
movies_df.drop(columns='tagline', axis=1, inplace=True)

In [59]:

# Transformamos la columna tutle para no crear conflictos con al API
movies_df['title'] = movies_df['title'].str.lower()

# Transformamos las columnas a float para no crear conflictos con la API
movies_df['release_year'] = movies_df['release_year'].astype(int)
movies_df['vote_count'] = movies_df['vote_count'].astype(int)
movies_df['id'] = movies_df['id'].astype(int)

# Redondear columnas a 2 decimales
movies_df['return'] = movies_df['return'].round(2)
movies_df['popularity'] = movies_df['popularity'].round(2)

# Reiniciamos el index
movies_df = movies_df.reset_index(drop=True)

In [60]:
# Resumen final del DataFrame movies_df 
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44026 entries, 0 to 44025
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   budget                   44026 non-null  float64       
 1   id                       44026 non-null  int64         
 2   original_language        44017 non-null  object        
 3   overview                 44026 non-null  object        
 4   popularity               44026 non-null  float64       
 5   release_date             44026 non-null  datetime64[ns]
 6   revenue                  44026 non-null  float64       
 7   runtime                  44026 non-null  float64       
 8   title                    44026 non-null  object        
 9   vote_average             44026 non-null  float64       
 10  vote_count               44026 non-null  int64         
 11  release_year             44026 non-null  int64         
 12  return                   44026 n

                                    **Carga**                                       

In [61]:
# Definimos la ruta de la carpeta donde se almacenarán los archivos procesados de "movies" después del ETL:
movies = '../data_post_etl/movies/'

# Creamos la carpeta "movies" (y cualquier carpeta intermedia necesaria) en la ruta especificada si no existe:
os.makedirs(movies, exist_ok=True)

# Especificamos la ruta completa del archivo "movies_dataset_etl.parquet" dentro de la carpeta "movies":
ruta = os.path.join(movies, 'movies_dataset_etl.parquet')

# Guardamos el DataFrame "movies_df" en formato Parquet en la ruta especificada, usando el motor "pyarrow" y sin incluir el índice:
movies_df.to_parquet(ruta, engine='pyarrow', index=False)

In [62]:
# Cargamos las nuevas tablas creadas

# Especificamos la ruta completa del archivo "genres.parquet" dentro de la carpeta "movies":
ruta2 = os.path.join(movies, 'genres.parquet')

# Guardamos el DataFrame "genres" en formato Parquet en la ruta especificada, usando el motor "pyarrow" y sin incluir el índice:
genres.to_parquet(ruta2, engine='pyarrow', index=False)